In [1]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [2]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. 
    #Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [3]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [4]:
import numpy as np
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [5]:
# activation function and its derivative
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def sigmoid_prime(x):
    #sigm = 1. / (1. + np.exp(-x))
    #return sigm * (1. - sigm)
    return (np.exp(-x) / (1+np.exp(-x))**2)

In [6]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)
        cost_ = []
        myerr=[]
        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                myerr.append(error)
            # calculate average error on all samples
            cost_.append(err)
            err /= samples
            
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
        return cost_ , myerr



In [7]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [11]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  8 22:52:57 2019

@author: moi
"""

import numpy as np



from keras.datasets import mnist
from keras.utils import np_utils




# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()


print(x_train[2])
from matplotlib import pyplot
for i in range(9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(x_train[i], cmap=pyplot.get_cmap('gray'))
    pyplot.show()


# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

print(y_train[2])
y_train = np_utils.to_categorical(y_train)

print(y_train[2])

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)







# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
cost_, myerr=net.fit(x_train[0:1000], y_train[0:1000], epochs=10, learning_rate=0.1)
cost2_, myerr2=net.fit(x_train[0:1000], y_train[0:1000], epochs=50, learning_rate=0.1)
#cost3_, myerr3=net.fit(x_train[0:1000], y_train[0:1000], epochs=10, learning_rate=2)

# Network2
# =============================================================================
net2 = Network()
net2.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net2.add(ActivationLayer(tanh, tanh_prime))
net2.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net2.add(ActivationLayer(tanh, tanh_prime))
net2.add(FCLayer(50, 25))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net2.add(ActivationLayer(tanh, tanh_prime))
net2.add(FCLayer(25, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net2.add(ActivationLayer(tanh, tanh_prime))
# =============================================================================

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
# =============================================================================
net2.use(mse, mse_prime)
cost22_,err22=net2.fit(x_train[0:1000], y_train[0:1000], epochs=10, learning_rate=0.1)
# =============================================================================

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

# =============================================================================
import matplotlib.pyplot as plt
plt.plot(cost_,'b')
plt.plot(cost22_,'r')
plt.plot(cost2_,'g')
# =============================================================================


ImportError: Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "C:\Users\LGHAOUCH\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 20, in swig_import_helper
   

In [9]:
import keras
import matplotlib.pyplot as plt
%matplotlib inline
def load_dataset(flatten=False):
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    # normalize x
    X_train = X_train.astype(float) / 255.
    X_test = X_test.astype(float) / 255.
    # we reserve the last 10000 training examples for validation
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]
    if flatten:
        X_train = X_train.reshape([X_train.shape[0], -1])
        X_val = X_val.reshape([X_val.shape[0], -1])
        X_test = X_test.reshape([X_test.shape[0], -1])
    return X_train, y_train, X_val, y_val, X_test, y_test
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=True)
## Let's look at some example
plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title("Label: %i"%y_train[i])
    plt.imshow(X_train[i].reshape([28,28]),cmap='gray');

ImportError: Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "C:\Users\LGHAOUCH\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 20, in swig_import_helper
   

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist


class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # calcule la sortie Y d'une couche pour une entrée X donnée
    def forward_propagation(self, input):
        raise NotImplementedError

    # calcule dE/dX pour un dE/dY donné (et met à jour les paramètres)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY.
    #Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error


class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)
        cost_ = []
        myerr = []
        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)

                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                myerr.append(error)
            # calculate average error on all samples
            cost_.append(err)
            err /= samples

            print('epoch %d/%d   error=%f' % (i + 1, epochs, err))
        return cost_, myerr
    pass

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x) ** 2

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def sigmoid_prime(x):
    # sigm = 1. / (1. + np.exp(-x))
    # return sigm * (1. - sigm)
    return (np.exp(-x) / (1 + np.exp(-x)) ** 2)

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

train_size = 0.7
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_seed=2019)

net = Network()
net.add(FCLayer(28 * 28, 100))  # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))  # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))  # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
cost_, myerr = net.fit(x_train[0:1000], y_train[0:1000], epochs=10, learning_rate=0.1)
cost2_, myerr2 = net.fit(x_train[0:1000], y_train[0:1000], epochs=50, learning_rate=0.1)
# cost3_, myerr3=net.fit(x_train[0:1000], y_train[0:1000], epochs=10, learning_rate=2)

# Network2
# =============================================================================
net2 = Network()
net2.add(FCLayer(28 * 28, 100))  # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net2.add(ActivationLayer(tanh, tanh_prime))
net2.add(FCLayer(100, 50))  # input_shape=(1, 100)      ;   output_shape=(1, 50)
net2.add(ActivationLayer(tanh, tanh_prime))
net2.add(FCLayer(50, 25))  # input_shape=(1, 100)      ;   output_shape=(1, 50)
net2.add(ActivationLayer(tanh, tanh_prime))
net2.add(FCLayer(25, 10))  # input_shape=(1, 50)       ;   output_shape=(1, 10)
net2.add(ActivationLayer(tanh, tanh_prime))
# =============================================================================

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
# =============================================================================
net2.use(mse, mse_prime)
cost22_, err22 = net2.fit(x_train[0:1000], y_train[0:1000], epochs=10, learning_rate=0.1)
# =============================================================================

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

# =============================================================================
import matplotlib.pyplot as plt

plt.plot(cost_, 'b')
plt.plot(cost22_, 'r')
plt.plot(cost2_, 'g')
# =============================================================================

ImportError: Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "C:\Users\LGHAOUCH\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 20, in swig_import_helper
   

In [5]:
pip install tensorflow==2.10.0

                                            0.3/455.9 MB 12.3 kB/s eta 10:19:44
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\AppData\Roaming\Python\Python38\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Users\LGHAOUCH\AppData\Roaming\Python\Python38\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\LGHAOUCH\AppData\Roaming\Python\Python38\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\LGHAOUCH\anaconda3\lib\http\client.py", line 458, in read
    n = self.readinto(b)
  File "C:\Users\LGHAOUCH\anaconda3\lib\http\client.py", line 502, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\LGHAOUCH\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\LGHAOUCH\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\LGHAOUCH\anaconda3

In [3]:
import tensorflow as tf
print(tf. __version__)

ImportError: Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "C:\Users\LGHAOUCH\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "C:\Users\LGHAOUCH\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 20, in swig_import_helper
   